In [267]:
import pymysql
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# DB 연결 - db 정보 업데이트 필요
connection = pymysql.connect(host='3.38.252.50', port=50266, user='root', password='asd456',
                       db='chatbotBase', charset='utf8')

# db에서 데이터 추출하여 데이터프레임으로 저장
with connection.cursor() as cursor:
    sql = "SELECT uuid, url, course_data, course, categorie, keyword, course_info FROM chat_bot220"
    cursor.execute(sql)  
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows)
#     print(df.head())

connection.close()

# 전체 컬럼명
['uuid', 'url', 'course_data', 'categorie_data', 'keyword_data', 'course_info_data', 'grade_data', 'stu_num_data', 'teacher_data', 'playtime_data', 'level_data', 'price_data', 'review_data', 'dibs_num_data', 'teacher_info_data', 'course', 'categorie', 'keyword', 'course_info', 'grade', 'stu_num', 'teacher', 'playtime', 'level', 'price', 'review', 'dibs_num', 'teacher_info', 'ori_data']

In [268]:
# 데이터프레임 컬럼명 변경
df.columns = ['uuid', 'url', 'course_data', 'course', 'categorie', 'keyword', 'course_info']

In [402]:
df2 = df.copy()

In [403]:
# 계획
# 강좌명, 카테고리, 키워드를 각각 df3, df4, df5, df6에 담은 후 함수를 통해 입력된 문장과 유사도가 높은 순서대로 내림차순 해주고 
# 4 곳에서 각각의 랭킹 점수를 매겨서 합산 점수 상위 5개의 강의 인덱스를 반환한다

In [404]:
df3 = df2[['uuid', 'course']].copy()
df4 = df2[['uuid', 'categorie']].copy()
df5 = df2[['uuid', 'keyword']].copy()
df6 = df2[['uuid', 'course_info']].copy()

In [405]:
# TfidfVectorizer 로 문서 벡터화
vectorizer = TfidfVectorizer()
vectors4 = vectorizer.fit_transform(df6['course_info']) # 1360
vectors = vectorizer.transform(df3['course']) # 1172
vectors2 = vectorizer.transform(df4['categorie']) # 58
vectors3 = vectorizer.transform(df5['keyword']) # 412

In [406]:
# 데이터프레임에 벡터 보관용 컬럼 추가
df3['vectors'] = list(vectors.toarray())
df4['vectors'] = list(vectors2.toarray())
df5['vectors'] = list(vectors3.toarray())
df6['vectors'] = list(vectors4.toarray())

In [407]:
# input sentence와 제일 유사한 아이템 내림차순 정렬 함수
def get_similar_documents(dataframe, input_sentence):
    # input sentence 벡터화
    input_vector = vectorizer.transform([input_sentence])

    # input vector와 입력된 DataFrame의 vectors 값 유사도 계산
    dit = dict()
    for idx, x in enumerate(cosine_similarity(input_vector.toarray(), dataframe['vectors'].tolist())[0]):
        dit[idx] = x
        
    # 유사도 높은 순으로 데이터프레임 내림차순 정렬
    dataframe = dataframe.iloc[[x[0] for x in sorted(dit.items(), key=lambda x : x[1], reverse=True)]]
    return dataframe

In [408]:
# 랭킹 점수 부여 함수
def dataframe_rank(df):
    df['rank'] = 0
    for i in range(len(df)):
        df['rank'][i] = len(df) - i
    return df

In [409]:
# get_similar_documents 함수 적용
# input_sentence: 입력값 df3: 강좌명, df4: 카테고리, df5: 키워드, df6: 강의 정보
input_sentence = "R 데이터 사이언스 심화"
df3 = get_similar_documents(df3, input_sentence)
df4 = get_similar_documents(df4, input_sentence)
df5 = get_similar_documents(df5, input_sentence)
df6 = get_similar_documents(df6, input_sentence)

df3.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)
df5.reset_index(drop=True, inplace=True)
df6.reset_index(drop=True, inplace=True)

In [414]:
# 랭킹 점수 부여 함수 적용
df3 = dataframe_rank(df3)
df4 = dataframe_rank(df4)
df5 = dataframe_rank(df5)
df6 = dataframe_rank(df6)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2984\1054700009.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'][i] = len(df) - i


In [411]:
# 강좌명, 카테고리, 키워드, 강의 정보 랭킹 점수 합산

df3.sort_values(by='uuid', inplace=True)
df4.sort_values(by='uuid', inplace=True)
df5.sort_values(by='uuid', inplace=True)
df6.sort_values(by='uuid', inplace=True)

df3.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)
df5.reset_index(drop=True, inplace=True)
df6.reset_index(drop=True, inplace=True)

df_ranked = df2[['uuid', 'url', 'course_data']].copy()
df_ranked['rank_course'] = df3['rank']
df_ranked['rank_category'] = df4['rank']
df_ranked['rank_keyword'] = df5['rank']
df_ranked['rank_course_info'] = df6['rank']

df_ranked['rank_sum'] = df_ranked['rank_course'] + df_ranked['rank_category'] + df_ranked['rank_keyword'] + df_ranked['rank_course_info']
df_ranked.sort_values(by='rank_sum', ascending=False, inplace=True)

In [412]:
# output: uuid, 강의명, url
output = df_ranked[['uuid', 'course_data', 'url']][:5]

In [413]:
output

,uuid,course_data,url
87,0c2167a32974025179aa0b6adbf76ea08e9f2ba0a03d88...,캐글 실습으로 배우는 데이터 사이언스,https://www.inflearn.com/course/데이터-사이언스-kaggle
143,14eeee3c7a76039910a6bcabb4890631c5c1308567631e...,데이터 활용의 시작 - 데이터 이해와 시각화 기초,https://www.inflearn.com/course/데이터활용-이해-시각화-기초
106,0ee2c3340b41bad6e9b7bce8ffbb8bcc43110c1ec05489...,"ELK 스택 (ElasticSearch, Logstash, Kibana) 으로 데이...",https://www.inflearn.com/course/elk-스택-데이터-분석
30,03b26e73b5f345b10564722bcc63199d6ab0837dd515bc...,"핵심만 빠르게, 입문자를 위한 파이썬(Python)과 판다스(Pandas)",https://www.inflearn.com/course/핵심-입문자-파이썬-판다스
74,0a18d297ba3ee52f18291ae0f5c96f47bd33783dcefafc...,R 프로그램 - 시즌2,https://www.inflearn.com/course/r-프로그래밍-시즌2
